# NOAA Sea Lion Population Count Challenge
This notebook is a cnn solution built off of previous approaches using keras. The images are resized conformly to 512x512. 

In [1]:
import keras
import cv2
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D, Input, concatenate
from keras.models import Model
from keras import backend as K
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


## Image Preprocessing
There needs to be preprocessing to reduce the size of images to fit into memory. This notebook assumes the images are all resized to 512x512. This can be done using ImageMagik

```sh
mogrify -resize 512x512! -format png -path test_images_512x512 Test/*.jpg
```

<hr>

## Configuration

In [2]:
n_classes= 5                   # amount of lion classes
batch_size= 16                 #
epochs= 100                    # number of epochs (start @ 100) 150 better score
image_size= 512                # resized img 
model_name= 'cnn_regression'   # 

In [3]:
'''
Return a list of the miss classed images to remove
'''
def read_ignore_list():
    df_ignore = pd.read_csv('../data/miss_class.txt')
    return df_ignore['train_id'].tolist()

In [4]:
'''
Removing mismatched images, reading in images w/ corresponding attr
'''
def load_data(dir_path):
    df = pd.read_csv('../data/train.csv')
    
    ignore_list = read_ignore_list()
    n_train_images = 948
    
    image_list = []
    y_list = []
    
    for i in range(0, n_train_images):
        if i not in ignore_list:
            img_path = os.path.join(dir_path, str(i) + '.png')
            img = cv2.imread(img_path)
            print('Image shape: ' + str(img.shape))
            
            image_list.append(img)
            
            row = df.ix[i]
            
            y_row = np.zeros((5))
            y_row[0] = row['adult_males']
            y_row[1] = row['subadult_males']
            y_row[2] = row['adult_females']
            y_row[3] = row['juveniles']
            y_row[4] = row['pups']
            y_list.append(y_row)
    x_train = np.asarray(image_list)
    y_train = np.asanyarray(y_list)
    
    print('X Train: ' + str(x_train.shape))
    print('Y Train: ' + str(y_train.shape))

    return x_train, y_train

## Model

In [5]:
'''
Original model
'''
def get_model():
    input_shape = (image_size, image_size, 3)
    
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size = (3, 3),
                    padding = 'same', input_shape = input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Conv2D(64, kernel_size = (3, 3),
                    padding = 'same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, kernel_size = (3, 3),
                    padding = 'same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Conv2D(n_classes, kernel_size=(3, 3), 
                     padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(GlobalAveragePooling2D())
    
    print(model.summary())
    
    model.compile(loss = keras.losses.mean_squared_error,
                  optimizer = keras.optimizers.Adadelta())
    return model

In [6]:
'''
Original model w/ dropout layers
'''
def get_dropout_model():
    input_shape = (image_size, image_size, 3)
    
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size = (3, 3),
                    padding = 'same', input_shape = input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Conv2D(64, kernel_size = (3, 3),
                    padding = 'same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(128, kernel_size = (3, 3),
                    padding = 'same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(n_classes, kernel_size=(3, 3), 
                     padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    model.add(GlobalAveragePooling2D())
    
    print(model.summary())
    
    model.compile(loss = keras.losses.mean_squared_error,
                  optimizer = keras.optimizers.Adadelta())
    return model

## Training

In [7]:
def train():
    # model = get_model()
    model = get_dropout_model()  # best
    
    # data -- resize to 512x512
    x_train, y_train = load_data('../data_512/')
        
    #, target_size=(512, 512)
    datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
    
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), 
                        steps_per_epoch = len(x_train) / batch_size, epochs = epochs)
    
    model.save('./models/' + model_name + '_model_Dropout.h5')

In [8]:
train()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 512, 512, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 512, 512, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      18496     
_________________________________________________________________
activation_2 (Activation)    (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 64)      0         
__________

/home/bss/opt/anaconda3/envs/noaa/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
I

Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
I

Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
Image shape: (512, 512, 3)
I

56/55 [==============================] - 17s - loss: 1828.1803    
Epoch 15/100
56/55 [==============================] - 17s - loss: 1819.7575    
Epoch 16/100
56/55 [==============================] - 17s - loss: 1835.7817    
Epoch 17/100
56/55 [==============================] - 17s - loss: 1827.8940    
Epoch 18/100
56/55 [==============================] - 17s - loss: 1823.2649    
Epoch 19/100
56/55 [==============================] - 17s - loss: 1813.8426    
Epoch 20/100
56/55 [==============================] - 17s - loss: 1819.6081    
Epoch 21/100
56/55 [==============================] - 17s - loss: 1827.4355    
Epoch 22/100
56/55 [==============================] - 17s - loss: 1840.1378    
Epoch 23/100
56/55 [==============================] - 17s - loss: 1815.4963    
Epoch 24/100
56/55 [==============================] - 17s - loss: 1827.5007    
Epoch 25/100
56/55 [==============================] - 17s - loss: 1816.2924    
Epoch 26/100
56/55 [==============================] -

## Submission

In [9]:
def create_submission():
    model = load_model('./models/' + model_name + '_model_Dropout.h5')
    
    n_test_images = 18636
    
    pred_arr = np.zeros((n_test_images, n_classes), np.int32)
    
    for k in range(0, n_test_images):
        image_path = '../data_512/' + str(k) + '.png'
        
        img = cv2.imread(image_path)
        img = img[None, ...]
        pred = model.predict(img).astype(int)
        
        pred_arr[k,:] = pred
    
    print('Pred arr: ' + str(pred_arr.shape))
    
    pred_arr = pred_arr.clip(min=0)
    
    df_submission = pd.DataFrame()
    df_submission['test_id'] = range(0, n_test_images)
    df_submission['adult_males'] = pred_arr[:,0]
    df_submission['subadult_males'] = pred_arr[:,1]
    df_submission['adult_females'] = pred_arr[:,2]
    df_submission['juveniles'] = pred_arr[:,3]
    df_submission['pups'] = pred_arr[:,4]
    
    df_submission.to_csv('./models/' + model_name + '_submission_dropout.csv', index = False)

In [10]:
create_submission()

Pred arr: (18636, 5)


## Resources

https://www.kaggle.com/c/noaa-fisheries-steller-sea-lion-population-count/

https://www.kaggle.com/radustoicescu/use-keras-to-classify-sea-lions-0-91-accuracy

https://www.kaggle.com/radustoicescu/count-the-sea-lions-in-the-first-image

https://github.com/mrgloom/Kaggle-Sea-Lions-Solution/blob/master/01-Image%20level%20regression/run_me.py